In [ ]:
library(tidyverse)
library(feather)
library(viridis)
library(flowMeans)
library(FlowSOM)

In [ ]:
set.seed(123)

In [ ]:
df1 <- read_feather('data/xs1.feather')

In [ ]:
colnames(df1)

In [ ]:
df <- df1 %>% 
select(matches('(CD|Ig|HLA).*'))

In [ ]:
dim(df)

### K-means 

In [ ]:
kmeans.df <- kmeans(df, centers=40)

In [ ]:
pca.df <- prcomp(df, center=TRUE, scale=TRUE, rank=2)

In [ ]:
ggplot(as.data.frame(pca.df$x), 
       aes(x=PC1, y=PC2, 
           color=as.factor(kmeans.df$cluster))) +
geom_point(size=1) +
guides(color=FALSE)

In [ ]:
colnames(df)

In [ ]:
ggplot(df, 
       aes(x=CD4_BUV661, y=CD8_PerCP.Cy55, 
           color=as.factor(kmeans.df$cluster))) +
geom_point(size=1) +
guides(color=FALSE)

#### Sanity check

In [ ]:
unif <- matrix(runif(20000,0,1),10000,2)

In [ ]:
dim(unif)

In [ ]:
kmeans.data <- kmeans(unif, centers=40)

In [ ]:
ggplot(as.data.frame(unif), 
       aes(x=V1, y=V2, 
           color=as.factor(kmeans.data$cluster))) +
geom_point(size=1) +
guides(color=FALSE)

### Using flowMeans: A flow-specific cluster method

In [ ]:
flowMeans.df <- flowMeans(df, MaxN = 20)

In [ ]:
summary(flowMeans.df)

In [ ]:
head(pca.df$x)

In [ ]:
data <- as.data.frame(pca.df$x)

ggplot(data, 
       aes(x=PC1, y=PC2, color=as.factor(flowMeans.df@Label))) +
geom_point(size=1) +
guides(color=FALSE)

#### Sanity check

In [ ]:
flowMeans.unif <- flowMeans(unif)

In [ ]:
summary(flowMeans.unif)

In [ ]:
ggplot(as.data.frame(unif), 
       aes(x=V1, y=V2, 
           color=as.factor(flowMeans.unif@Label))) +
geom_point(size=1) +
guides(color=FALSE)